In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Hackathon/superhero.csv")
df=df.rename(columns={'jumpa':'name'})
df

,name
0,doctor fate
1,starlight
2,isildur
3,lasher
4,varvara
...,...
9047,sidestep
9048,proctor
9049,sonny sumo
9050,super sonic


In [3]:
t= list(df['name'])
corpus= '\n'.join(str(i) for i in t)

In [4]:
corpus

'doctor fate\nstarlight\nisildur\nlasher\nvarvara\nthe target\naxel\nbattra\nchangeling\npyrrha\nthe rhino\nmajor glory\ndrago\nking vegeta\nphoebus apollo\nwarpath\nthe lost sapphire\ndurok the demolisher\ncronos\ndragon warrior\nblaziken\nmighty destroyer\nthe firedrake of gondolin\nking kid\nstar boy\ndarth plagueis\nazrel\nhulk\ndark gaia phoenix\nklaw\nmichael demiurgos\nshilo norman\nspiderhulk\nzauriel\ndevo\nhypnotia\nqueen rhea\nfanor\nscourge\nxerneas\nbenjamin sisko\noutsiders\ncyclone\nking kong\nchronos\nsyn shenron\ncrimson chin\ncyborg\nhorsewoman\nbryan mills\nlothric  lorian\nsingularity\nblaise zabini\nsoldier \nmutant leader\nsymbiote electro\ndeath of death\ncyborg\nhiruzen sarutobi\navengelyne\nquicksilver\nnymphadora tonks\nkulan gath\nthe punisher\ntyrion lannister\npgell\ndamien darhk\nsquirrel girl\nras al ghul\nflash dr manhattan\nlivewire\nblizzard of hell\nconcrete\ngamabunta\nslimani\nnien nunb\nlilandra\nanarky\nbaron von blitzschlag\nlord farquaad\nleman 

In [5]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [6]:
tokenizer.fit_on_texts(corpus)

In [7]:
vocab=len(tokenizer.word_index.items())+1
tokenizer.word_index.items()

dict_items([('a', 1), ('e', 2), ('r', 3), ('o', 4), ('n', 5), ('i', 6), (' ', 7), ('t', 8), ('s', 9), ('l', 10), ('m', 11), ('h', 12), ('d', 13), ('c', 14), ('u', 15), ('g', 16), ('k', 17), ('b', 18), ('p', 19), ('y', 20), ('w', 21), ('f', 22), ('v', 23), ('j', 24), ('z', 25), ('x', 26), ('q', 27)])

In [8]:
tokenizer.texts_to_sequences("doctor")

[[13], [4], [14], [8], [4], [3]]

In [9]:
encoded=[]
for word in df['name']:
  s=tokenizer.texts_to_sequences(word)
  s=np.array(s).reshape(len(s))
  encoded.append(list(s))


In [ ]:
encoded

In [11]:
max_len=max([len(x) for x in encoded])
print(max_len)

32


In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_encoded=pad_sequences(encoded,padding="pre",maxlen=max_len)
pad_encoded

array([[ 0,  0,  0, ...,  1,  8,  2],
       [ 0,  0,  0, ..., 16, 12,  8],
       [ 0,  0,  0, ..., 13, 15,  3],
       ...,
       [ 0,  0,  0, ..., 15, 11,  4],
       [ 0,  0,  0, ...,  5,  6, 14],
       [ 0,  0,  0, ...,  4, 19,  9]], dtype=int32)

In [13]:
x=pad_encoded[:,0:max_len-1]
y=pad_encoded[:,max_len-1]

In [14]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=vocab)

In [15]:
len(x[0])

31

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
model=Sequential()
model.add(LSTM(100,input_shape=(max_len-1,1)))
model.add(Dense(50,activation="relu"))
model.add(Dense(vocab,activation="softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [18]:
model.fit(x,y,epochs=20)

Epoch 1/20
283/283 [==============================] - 11s 5ms/step - loss: 2.6965 - accuracy: 0.2140
Epoch 2/20
283/283 [==============================] - 1s 5ms/step - loss: 2.5634 - accuracy: 0.2335
Epoch 3/20
283/283 [==============================] - 1s 5ms/step - loss: 2.5173 - accuracy: 0.2396
Epoch 4/20
283/283 [==============================] - 1s 5ms/step - loss: 2.4764 - accuracy: 0.2439
Epoch 5/20
283/283 [==============================] - 1s 5ms/step - loss: 2.4336 - accuracy: 0.2699
Epoch 6/20
283/283 [==============================] - 1s 5ms/step - loss: 2.3922 - accuracy: 0.2831
Epoch 7/20
283/283 [==============================] - 1s 5ms/step - loss: 2.3461 - accuracy: 0.3047
Epoch 8/20
283/283 [==============================] - 1s 5ms/step - loss: 2.2997 - accuracy: 0.3230
Epoch 9/20
283/283 [==============================] - 1s 5ms/step - loss: 2.2526 - accuracy: 0.3354
Epoch 10/20
283/283 [==============================] - 1s 5ms/step - loss: 2.2098 - accuracy: 0.353

In [19]:
# seed=input("enter a word")
# seed_seq=tokenizer.texts_to_sequences(seed)
# encoded_seed=(np.array(seed_seq).reshape(1,len(seed_seq)))
# pad_encoded_seed=pad_sequences(encoded_seed,padding="pre",maxlen=max_len-1)

In [20]:
# pad_encoded_seed

In [21]:
# ypreds=model.predict(pad_encoded_seed)
# next=np.argmax(ypreds)
# next

In [22]:
# ypreds

In [23]:
def generator(tokenizer,model,num):
  seed=input("enter a word")
  for i in range (0,num):
    seed_seq=tokenizer.texts_to_sequences(seed)
    encoded_seed=(np.array(seed_seq).reshape(1,len(seed_seq)))
    pad_encoded_seed=pad_sequences(encoded_seed,padding="pre",maxlen=max_len-1)
    ypreds=model.predict(pad_encoded_seed)
    next_index=np.argmax(ypreds)
    for (k,v) in tokenizer.word_index.items():
      if v==next_index:
        next_char=k
        break
    seed=seed+next_char
  return seed


In [24]:
num=int(input("enter the number of next characters"))
generator(tokenizer,model,num)

enter the number of next characters5
enter a wordnaxal
1/1 [==============================] - 0s 16ms/step


'naxalernan'